In [1]:
# import the opencv library
import cv2
from pytorch_helper import PyTorchHelper
from viewer import Viewer

In [2]:
viewer = Viewer()

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from torchvision.models import resnet18
import math

In [4]:
model = torch.jit.load('output/the_best.pt')

In [5]:
helper = PyTorchHelper(0,None)

In [6]:
import torchvision.transforms as T
img_size = (256, 256)
train_transforms = T.Compose([T.Resize(img_size),
                              T.ToTensor()])

In [7]:
def predict(frame):
    height, width, depth = frame.shape
    img = viewer.convert_from_cv2_to_image(frame)
    x_data = train_transforms(img)
    pred = model(x_data.unsqueeze(0))[0]
    if torch.round(torch.sigmoid(pred[0])) == 1:
        x_top_left = math.ceil(pred[1] * width)
        y_top_left = math.ceil(pred[2] * height)

        x_bottom_right = math.ceil(pred[3] * width)
        y_bottom_right = math.ceil(pred[4] * height)

        frame = cv2.rectangle(frame, (x_top_left, y_top_left), (x_bottom_right, y_bottom_right),
                                (0, 0, 255), 4)
    return frame    

In [8]:

  
  
# define a video capture object
vid = cv2.VideoCapture(0)
try:
    while(True):

        # Capture the video frame
        # by frame
        ret, frame = vid.read()

        # Display the resulting frame
        frame = predict(frame)
        cv2.imshow('frame', frame)

        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:  
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()